<a href="https://colab.research.google.com/github/LCaravaggio/AnalisisCuantitativoAvanzado/blob/main/SeriesdeTiempo/EMAE_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import json

import requests
from requests.auth import HTTPBasicAuth

import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/alphacast.json", 'r') as f:
    api_token= json.load(f)

Mounted at /content/drive


In [3]:
# Definir user key para la API
user_key = api_token["key"]

In [4]:
# EMAE
emae = requests.get('https://api.alphacast.io/datasets/5331/data?apiKey='+user_key+'&&$format=csv', auth=HTTPBasicAuth(user_key, ''))
open('emae.csv','wb').write(emae.content)
emae = pd.read_csv('emae.csv',delimiter = ',')

In [8]:
emae.set_index('Date', inplace = True)
emae = emae.loc['2016-02-01':]
y=emae['Emae - current_prices_mom']

In [10]:
# IPI
ipi = requests.get('https://api.alphacast.io/datasets/5679/data?apiKey='+user_key+'&&$format=csv', auth=HTTPBasicAuth(user_key, ''))
open('ipi.csv','wb').write(ipi.content)
ipi = pd.read_csv('ipi.csv',delimiter = ',')


ipi['Date'] = pd.to_datetime(ipi['Date'])
ipi.set_index('Date', inplace = True)
ipi = ipi[(ipi['Division'] == "IPI Manufacturero") & (ipi['Seasonal Adjustment'] == "Seasonally Adjusted")]
ipi = ipi['Value - sa_MoM']
ipi.columns = ['Value - sa_MoM']
ipi = ipi.loc['2016-02-01':]

In [12]:
# ISAC
isac = requests.get('https://api.alphacast.io/datasets/5565/data?apiKey='+user_key+'&&$format=csv', auth=HTTPBasicAuth(user_key, ''))
open('isac.csv','wb').write(isac.content)
isac = pd.read_csv('isac.csv',delimiter = ',')


isac['Date'] = pd.to_datetime(isac['Date'])
isac.set_index('Date', inplace = True)
isac = isac['Nivel general - sa_MoM']
isac.columns = ['Nivel general - sa_MoM']
isac = isac.loc['2016-02-01':]

In [14]:
isac=pd.Series(isac, name='ISAC')
ipi=pd.Series(ipi, name='IPI')

In [15]:
X = pd.concat([ipi, isac], axis=1)

In [19]:
from sklearn import linear_model
lr = linear_model.LinearRegression()
lr.fit(X[:-1], y)
Yest = lr.predict(X)

In [26]:
Yest

array([ 2.05632676e-01, -4.20358996e-01, -8.44221140e-01,  6.13662608e-01,
       -3.85649888e-01, -2.65928043e-01,  1.48173567e+00,  1.19423475e-01,
       -1.67960140e+00,  2.01405643e+00,  6.73385174e-01,  1.72370129e-01,
       -4.33332606e-02,  1.61849256e+00, -9.12171220e-01,  5.86282476e-01,
        7.73274975e-01, -2.22072089e-01,  4.80957791e-01,  8.96772855e-02,
        5.93984144e-01,  9.32867428e-01, -4.49127272e-01,  6.35908365e-01,
       -2.35458147e-01,  3.03464457e-01, -1.96351602e-01, -2.22397704e-01,
        5.79804033e-02, -1.68834597e-01,  3.54650801e-01, -1.87397179e-01,
       -4.31607562e-02, -6.75866830e-01, -7.57139643e-01,  1.21143848e+00,
        1.53970113e+00, -7.95692809e-01,  4.09332794e-01,  4.32968413e-01,
       -1.35833105e+00,  1.18537410e+00, -1.48298585e-01, -5.52762804e-01,
       -3.53524168e-01,  4.30493655e-02, -8.79434691e-01, -2.24532704e-02,
        1.34620133e-01, -5.19773112e+00, -6.81588245e+00,  1.48511130e+01,
        7.40996572e+00,  

In [22]:
print('DATOS DEL MODELO REGRESIÓN LINEAL SIMPLE')
print()
print('Valor de la pendiente o coeficiente "a":')
print(lr.coef_)
print('Valor de la intersección o coeficiente "b":')
print(lr.intercept_)
print()
print('La ecuación del modelo es igual a:')
print('y = ', lr.coef_, 'X + ', lr.intercept_)
print()
print('Precisión del modelo:')
print(lr.score(X[:-1], y))

DATOS DEL MODELO REGRESIÓN LINEAL SIMPLE

Valor de la pendiente o coeficiente "a":
[-0.05085379  0.16227664]
Valor de la intersección o coeficiente "b":
0.11130067493589973

La ecuación del modelo es igual a:
y =  [-0.05085379  0.16227664] X +  0.11130067493589973

Precisión del modelo:
0.1528041309179733


In [23]:
lr.coef_[0]

-0.05085378629688597

In [24]:
emae_predict=X[-1:]['IPI']*lr.coef_[0]+X[-1:]['ISAC']*lr.coef_[1]+lr.intercept_

In [25]:
emae_predict

Date
2023-05-01    0.297174
dtype: float64